In [1]:
%matplotlib inline
import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

import re
import pickle
import pandas as pd
import csv
import os
import time
import datetime
from tensorflow.contrib import learn
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import f1_score
from IPython.display import clear_output, Image, display, HTML

In [2]:
###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
###### Do not modify  here ######

In [3]:
def load_data_and_labels(train_data_file, test_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    train_data = pd.read_csv(train_data_file, sep='\t', quoting=csv.QUOTE_NONE, header=None, names=['id', 'language', 'label', 'text'])
    test_data = pd.read_csv(test_data_file, sep='\t', quoting=csv.QUOTE_NONE, header=None, names=['id', 'language', 'label', 'text'])
    
    x_train = train_data['text'].tolist()
    y_train = train_data['label'].tolist()

    x_test = test_data['text'].tolist()
    y_test = test_data['label'].tolist()
    
    x_train = [s.strip() for s in x_train]
    x_test = [s.strip() for s in x_test]
    
    label_encoding = {'neutral':0, 'positive':1, 'negative':2}
    
    y_train_encoding = [label_encoding[label] for label in y_train]    
    y_test_encoding = [label_encoding[label] for label in y_test]

    
    return [x_train, y_train_encoding, x_test, y_test_encoding]

def transform_data_and_labels(data):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    
    x = np.array(data['text'].tolist())
    y = data['label'].tolist()
    
    # encoding label
    label_encoding = {'neutral':0, 'positive':1, 'negative':2}
    y = [label_encoding[label] for label in y]    
    
    
    y = np.array(y)
    
    # Randomly shuffle data
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    # Split train/test set
    # maybe we can use cross-validation to improve
    dev_sample_index = -1 * int(0.1 * float(len(y)))
    x_train, x_test = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_test = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]
    print("Train/Test split: {:d}/{:d}".format(len(y_train), len(y_test)))
    
    return [x_train, y_train, x_test, y_test]


"""
    This function assumes that the last word in the word embedding is a zero vector, and will use it as padding.
    The input 'num_voc' equals to the shape[0] of the word embedding.
"""
def process_tweet(train_tweets, test_tweets, num_voc):
    # max_document_length = max([len(x.split(" ")) for x in x_train_sentence])
    ppl_re = re.compile(r'@\S*')
    url_re = re.compile(r'http\S+')
    tknzr = TweetTokenizer()
    # tknzr = TweetTokenizer(reduce_len=True)
    
    tokenized_tweets_all = []
    max_document_length = 0
    
    for tweets in [train_tweets, test_tweets]:
        tweets = [url_re.sub('URLTOK', ppl_re.sub('USRTOK', tweet.lower())) for tweet in tweets]
        tokenized_tweets = [tknzr.tokenize(tweet) for tweet in tweets]
        tokenized_tweets_all.append(tokenized_tweets)
        max_document_length = max(max_document_length, max([len(tweet) for tweet in tokenized_tweets]))
    print(max_document_length)
    
    x = []
    
    for tokenized_tweets in tokenized_tweets_all:
        x_curr = []
        for tokenized_tweet in tokenized_tweets:
            if len(tokenized_tweet) == max_document_length:
                print(tokenized_tweet)
            """Not sure if original paper does this, but since index 0 means USRTOK, padding should be a number
            higher than total word count, so tf.nn.embedding_lookup will return a tensor of 0 insted of USRTOK."""
        #     temp = np.zeros(max_document_length, dtype=np.int).tolist()
            temp = (np.ones(max_document_length, dtype=np.int)*(num_voc-1)).tolist()

            for index, word in enumerate(tokenized_tweet):
                if word in word_dict:
#                     temp[index] = word_dict[word][0]
                    temp[index] = word_dict[word]
            x_curr.append(temp)
        x_curr = np.array(x_curr)
        x.append(x_curr)
    
    return x[0], x[1]


def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        print("Current epoch: ", epoch)
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

# Load pre-train word embeddings

In [18]:
# final_embeddings = np.load('./data/embed_tweets_en_200M_200D/embedding_matrix.npy')
# word_dict = {}
# with open('./data/embed_tweets_en_200M_200D/vocabulary.pickle', 'rb') as myfile:
#     word_dict = pickle.load(myfile)

In [4]:
final_embeddings = np.load('./data/embed_tweets_en_590M_52D_data/en_word2vec_52.npy')
word_dict = {}
with open('./data/embed_tweets_en_590M_52D_data/vocabulary_dict_52.pickle', 'rb') as myfile:
    word_dict = pickle.load(myfile)

In [5]:
# shit
for key, val in word_dict.items():
    if val == 94:
        print(key, val)

76.44 94


In [6]:
word_dict

{'@lauren_hp': 3568330,
 'hadcore': 1145459,
 '@oobeyme__': 2899054,
 '@literallylayna': 5024063,
 '@daisbond_': 372284,
 '@that_mann_mike': 3085518,
 '@liveasy10': 1524849,
 '@bikinimowing': 3013104,
 '@kianaparrr': 4325737,
 '@duhhasian18': 3054969,
 '#nayariveraontheellenshow': 203198,
 '@niditriyant': 3489693,
 '@solene_ldn': 4085061,
 '@moonmunch': 3419030,
 '@logical_saario': 4210626,
 '@tylermcflurry': 382946,
 '@ashleigh_kwele_': 3124617,
 '@joecrizzydchill': 1304721,
 '@josidschultz': 1920915,
 '#koanteykoti': 2494613,
 '@ms_baum': 3784082,
 '@mikejacques1': 1809177,
 '@kidlyznick1020': 1434302,
 '@cheyannemarieee': 5967408,
 '@yesigalano': 4104663,
 '@nabillasaurs': 3283274,
 '@shaylamarieee_': 3285170,
 '@dookie92': 2101334,
 "buzz'n": 667825,
 '@rockingstep': 830543,
 '@weerapreya': 3941575,
 '@le0nthedon': 5789832,
 'it.lots': 6085389,
 '@marcussskwan': 3511959,
 '@seto_balyan': 1060388,
 '@okikawasa': 3140443,
 '@xalisonskiba': 212137,
 '#kidrauhlliveson': 3104926,
 '@ohm

In [7]:
final_embeddings.shape

(6140853, 52)

# Distant Supervision phase

In [8]:
distance_supervised_tweets = pd.read_csv('./data/distant_data/distance_supervised_tweets_corrected', names=["id","lan", "label", "text"], sep="\t", header=None, usecols=["lan", "label", "text"])
distance_supervised_tweets_2 = pd.read_csv('./data/distant_data/distance_supervised_tweets_2_corrected', names=["id","lan", "label", "text"], sep="\t", header=None, usecols=["lan", "label", "text"])
distance_supervised_tweets_3 = pd.read_csv('./data/distant_data/distance_supervised_tweets_3_corrected', names=["id","lan", "label", "text"], sep="\t", header=None, usecols=["lan", "label", "text"])
distance_supervised_tweets = distance_supervised_tweets.append(distance_supervised_tweets_2).append(distance_supervised_tweets_3)

In [9]:
distance_supervised_tweets

,lan,label,text
0,en,positive,Boston Bruins Morning Thoughtefense Exceeding ...
1,en,positive,Bol Bachchan!. #AeZindagiGaleLagale. Aata Majh...
2,en,negative,"(17) karena lagi sakit, aku lagi gelisah terus..."
3,en,positive,Telkomsel: Thary422 Terima kasih telah berpart...
4,en,positive,@parisa_khania آخر سر هم از جزوه ی محترم عکس گ...
5,en,positive,kyristcl: XL123: frungnarikvnn Bisa ajak selai...
6,en,positive,Telkomsel: daff_01 Terima kasih telah berparti...
7,en,positive,@chris_randall Just the usual disclaimer that ...
8,en,positive,@Lin_Manuel Congrats from me and all my friend...
9,en,negative,"@nicaaaji hahahaha magtext ako beb. ""Hi lola! ..."


In [11]:
x_train_distance, y_train, x_test_distance, y_test = transform_data_and_labels(distance_supervised_tweets)


Train/Test split: 183161/20351


In [12]:
# TODO Load distant-supervised data
# train it with two-layer CNN model
# pass the weight to next two layer CNN model
print(len(x_test_sentence))
x_train_distance, x_test_distance = process_tweet(x_train_distance, x_test_distance, final_embeddings.shape[0])


20351
2063
['people', 'come', 'and', 'people', 'go', '...', "that's", 'life', '...', '#aldubmistakenidentity', '33877', 'en', 'positive', "michiganon't", 'sell', 'USRTOK', '100m', 'gallons', 'of', 'groundwater', 'for', '$', '200', 'and', '20', 'jobs', '.', "that's", 'bananas', '.', 'URLTOK', '33878', 'en', 'positive', 'jio', 'has', 'touched', 'the', 'hearts', 'of', '50', 'millions', 'users', 'with', 'their', 'network', 'really', 'happy', 'for', 'jio', '!', '#jio50million', '33879', 'en', 'positive', 'every', 'second', ',', 'minute', '&', 'hour', 'of', 'our', 'life', 'must', 'be', 'filled', 'with', 'passion', ',', 'dedication', 'and', 'restlessness', 'to', 'make', 'it', 'the', 'best', 'possible', 'life', 'ever', '.', '33880', 'en', 'positive', 'USRTOK', 'USRTOK', 'USRTOK', 'whys', 'this', 'yous', '33881', 'en', 'positive', 'thanks', 'for', 'the', 'recent', 'follow', 'USRTOK', 'USRTOK', 'USRTOK', 'happy', 'to', 'connect', 'have', 'a', 'great', 'tuesday', '.', '>', '>', 'URLTOK', '33882',

In [ ]:
# To save memory
# del x_train_sentence
# del x_test_sentence

In [13]:
# put Word2Vec on 590 million English Tweets using 52 dimensions.
vocabulary_size = final_embeddings.shape[0]
embedding_size = final_embeddings.shape[1]  # Dimension of the embedding vector.
graph = tf.Graph()

sequence_length=x_train_distance.shape[1]
num_classes=3

# filter_sizes: The number of words we want our convolutional filters to cover. 
# We will have num_filters for each size specified here. 
# For example, [3, 4, 5] means that we will have filters that slide over 3, 4 and 5 words respectively, for a total of 3 * num_filters filters.
first_filter_sizes = [4]
first_pool_window_sizes = [4]
first_pool_strides = [2]


second_filter_window_sizes = [3]
num_filters = 200

# No L2 norm
l2_reg_lambda=0.0

with graph.as_default():
    input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
    input_y = tf.placeholder(tf.int64, [None], name="input_y")
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(tf.constant(0.0, shape=[vocabulary_size, embedding_size]),
                        trainable=False, name="embedding")

        embedding_placeholder = tf.placeholder(tf.float32, [vocabulary_size, embedding_size], name='word_embedding_placeholder')
        embedding_init = embeddings.assign(embedding_placeholder)  # assign exist word embeddings

        embedded_chars = tf.nn.embedding_lookup(embeddings, input_x)
    embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)
    print(input_x.shape)
    print(embedded_chars_expanded.shape)
    # Keeping track of l2 regularization loss (optional)
    l2_loss = tf.constant(0.0)
    
     # Add variable initializer.
    init = tf.global_variables_initializer()
    
    
    # Create first cnn : a convolution + maxpool layer for each filter size    
    # 1st Convolution Layer
    for i, first_filter_size in enumerate(first_filter_sizes):
        with tf.name_scope("conv-maxpool-1"):
            # Convolution Layer
            filter_shape = [first_filter_size, embedding_size, 1, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            print("CNN filter", W.shape)
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(
                embedded_chars_expanded,
                W,
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="conv")
            # Apply nonlinearity
            h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            # Maxpooling over the outputs and transform for next layer, so the "channel" of convolution 
            # will become "input_width" for next layer
#             pooled = tf.transpose(tf.nn.max_pool(
#                 h,
#                 ksize=[1, first_pool_window_sizes[i], 1, 1],
#                 strides=[1, first_pool_strides[i], 1, 1],
#                 padding='VALID',
#                 name="pool"), perm=[0, 1, 3, 2])
            pooled = tf.nn.max_pool(
                h,
                ksize=[1, first_pool_window_sizes[i], 1, 1],
                strides=[1, first_pool_strides[i], 1, 1],
                padding='VALID',
                name="pool")
#     print("conv1", conv.shape)
#     print("h1", h.shape)
#     print("pooled1", pooled_1.shape)
    
    # 2nd Convolutional Layer
#     for i, second_filter_size in enumerate(second_filter_window_sizes):
#         with tf.name_scope("conv-maxpool-2"):
#             # Convolution Layer
#             filter_shape = [second_filter_size, num_filters, 1, num_filters]
#             W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
#             print("CNN filter", W.shape)
#             b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
#             conv = tf.nn.conv2d(
#                 pooled,
#                 W,
#                 strides=[1, 1, 1, 1],
#                 padding="VALID",
#                 name="conv")
#             # Apply nonlinearity
#             h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
#             # Maxpooling over the outputs and transform for next layer, so the "channel" of convolution 
#             # will become "input_width" for next layer
#             pooled = tf.nn.max_pool(
#                 h,
#                 ksize=[1, h.shape[1], 1, 1],
#                 strides=[1, 1, 1, 1],
#                 padding='VALID',
#                 name="pool")
    for i, second_filter_size in enumerate(second_filter_window_sizes):
        with tf.name_scope("conv-maxpool-2"):
            # Convolution Layer
            filter_shape = [second_filter_size, 1, num_filters, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            print("CNN filter", W.shape)
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(
                pooled,
                W,
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="conv")
            # Apply nonlinearity
            h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            print(h.shape)
            # Maxpooling over the outputs and transform for next layer, so the "channel" of convolution 
            # will become "input_width" for next layer
            pooled = tf.nn.max_pool(
                h,
                ksize=[1, h.shape[1], 1, 1],
                strides=[1, 1, 1, 1],
                padding='VALID',
                name="pool")
 

    h_pool_flat = tf.reshape(pooled, [-1, num_filters])  # flatten pooling layers
    print("h_pool_flat", h_pool_flat.shape)
    
    # Add dropout
#     with tf.name_scope("dropout"):
#         self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

    # Final (unnormalized) scores and predictions
    
    # Fully connected hidden layer
    with tf.name_scope("hidden"):
        with tf.variable_scope("hidden"):
            W = tf.get_variable(
                "W",
                shape=[num_filters, num_filters],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            out = tf.nn.relu(tf.nn.xw_plus_b(h_pool_flat, W, b))
        
    
    with tf.name_scope("output"):
        with tf.variable_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            scores = tf.nn.xw_plus_b(out, W, b, name="scores")
            print("scores", scores.shape)
            predictions = tf.argmax(scores, 1, name="predictions")
            print("predictions", predictions.shape)


    # Calculate mean cross-entropy loss
    with tf.name_scope("loss"):
        losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=scores, labels=input_y)
        print("losses", losses.shape)
        loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

    # Accuracy
    with tf.name_scope("accuracy"):
        correct_predictions = tf.equal(predictions, input_y)
        accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

(?, 2063)
(?, 2063, 52, 1)
CNN filter (4, 52, 1, 200)
CNN filter (3, 1, 200, 200)
(?, 1027, 1, 200)
h_pool_flat (?, 200)
scores (?, 3)
predictions (?,)
losses (?,)


In [15]:
batch_size = 512
num_epochs = 1

num_checkpoints = 5
print_train_every = 5
evaluate_every = 50
checkpoint_every = 10000000
allow_soft_placement=True
log_device_placement=False

with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=allow_soft_placement,
      log_device_placement=log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdadeltaOptimizer(1.0)
        grads_and_vars = optimizer.compute_gradients(loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", loss)
        acc_summary = tf.summary.scalar("accuracy", accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

#         # Write vocabulary
#         vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())


        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              input_x: x_batch,
              input_y: y_batch,
            }
            _, step, summaries, cur_loss, cur_accuracy, y_pred = sess.run(
                [train_op, global_step, train_summary_op, loss, accuracy, predictions],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
#             print(y_pred)
#             print(y_batch)
            if step % print_train_every == 0:
                f1 = f1_score(y_batch, y_pred, average = 'weighted')
                print("{}: step {}, loss {:g}, acc {:g}, f1 {:g}".format(time_str, step, cur_loss, cur_accuracy,
                                                                     f1))
            train_summary_writer.add_summary(summaries, step)



        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              input_x: x_batch,
              input_y: y_batch,
            }
            step, summaries, cur_loss, cur_accuracy, y_pred = sess.run(
                [global_step, dev_summary_op, loss, accuracy, predictions],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            f1 = f1_score(y_batch, y_pred, average = 'weighted')
            print("Test")
            print("{}: step {}, loss {:g}, acc {:g}, f1 {:g}".format(time_str, step, cur_loss, cur_accuracy,
                                                                    f1))
            if writer:
                writer.add_summary(summaries, step)
        
        def dev_step_batch(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              input_x: x_batch,
              input_y: y_batch,
            }
            step, summaries, cur_loss, cur_accuracy, y_pred = sess.run(
                [global_step, dev_summary_op, loss, accuracy, predictions],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            f1 = f1_score(y_batch, y_pred, average = 'weighted')
#             print("{}: step {}, loss {:g}, acc {:g}, f1 {:g}".format(time_str, step, cur_loss, cur_accuracy,
#                                                                     f1))
            if writer:
                writer.add_summary(summaries, step)
            return cur_loss, cur_accuracy, f1
        
        
        sess.run(embedding_init, feed_dict={embedding_placeholder: final_embeddings})
        # Generate batches
        batches = batch_iter(
            list(zip(x_train_distance, y_train)), batch_size, num_epochs)
        
        batches_test = list(batch_iter(
            list(zip(x_test_distance, y_test)), batch_size, 1))
        
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            
            current_step = tf.train.global_step(sess, global_step)
            if current_step % evaluate_every == 0:
                print("\nEvaluation:")
                total_loss=0
                total_f1=0
                total_accuracy=0
                len_of_batch = int(len(batches_test))
                for batch_test in batches_test:
                    x_batch_test, y_batch_test = zip(*batch_test)
                    cur_loss, cur_accuracy, cur_f1 = dev_step_batch(x_batch_test, y_batch_test, writer=dev_summary_writer)
                    total_loss+=cur_loss
                    total_accuracy+=cur_accuracy
                    total_f1+=cur_f1
                print("loss {:g}, acc {:g}, f1 {:g}".format(total_loss/len_of_batch, total_accuracy/len_of_batch, total_f1/len_of_batch))
                print("")
            if current_step % checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))
        final_embeddings = embeddings.eval()

INFO:tensorflow:Summary name conv-maxpool-1/W:0/grad/hist is illegal; using conv-maxpool-1/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-1/W:0/grad/sparsity is illegal; using conv-maxpool-1/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-1/b:0/grad/hist is illegal; using conv-maxpool-1/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-1/b:0/grad/sparsity is illegal; using conv-maxpool-1/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-2/W:0/grad/hist is illegal; using conv-maxpool-2/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-2/W:0/grad/sparsity is illegal; using conv-maxpool-2/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-2/b:0/grad/hist is illegal; using conv-maxpool-2/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-2/b:0/grad/sparsity is illegal; using conv-maxpool-2/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name hidden/W:0/grad/hist is ill

/home/phejimlin/anaconda3/envs/tensorflow_3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


2017-11-19T01:24:24.238155: step 5, loss 0.469793, acc 0.835938, f1 0.761237
2017-11-19T01:24:27.531674: step 10, loss 0.452523, acc 0.832031, f1 0.755747
2017-11-19T01:24:30.652735: step 15, loss 0.430573, acc 0.849609, f1 0.780528
2017-11-19T01:24:33.748674: step 20, loss 0.458488, acc 0.820312, f1 0.739337
2017-11-19T01:24:36.863587: step 25, loss 0.417459, acc 0.863281, f1 0.799938
2017-11-19T01:24:39.966193: step 30, loss 0.449762, acc 0.820312, f1 0.746475
2017-11-19T01:24:43.064461: step 35, loss 0.436679, acc 0.820312, f1 0.746535
2017-11-19T01:24:46.197001: step 40, loss 0.417404, acc 0.835938, f1 0.7684
2017-11-19T01:24:49.323127: step 45, loss 0.431263, acc 0.830078, f1 0.753006
2017-11-19T01:24:52.442455: step 50, loss 0.437058, acc 0.839844, f1 0.784143

Evaluation:
loss 0.441994, acc 0.831175, f1 0.754614

2017-11-19T01:25:04.992139: step 55, loss 0.509859, acc 0.8125, f1 0.728448
2017-11-19T01:25:08.146066: step 60, loss 0.392543, acc 0.851562, f1 0.78703
2017-11-19T01:2

In [16]:
final_embeddings

array([[-0.056935  ,  0.038956  ,  0.39726499, ..., -0.108269  ,
         0.242364  ,  0.238135  ],
       [-0.178036  ,  0.165739  ,  0.29103801, ..., -0.206843  ,
         0.133855  ,  0.193499  ],
       [-0.27868199, -0.081914  ,  0.222532  , ..., -0.30557701,
        -0.094965  ,  0.31291401],
       ..., 
       [-0.86243898, -0.106481  ,  0.77118099, ..., -0.72408199,
        -0.133205  ,  1.04446006],
       [-0.122979  ,  0.053025  ,  1.02682495, ...,  0.41790599,
        -0.55734599,  0.42464599],
       [-0.95815903,  0.023787  ,  1.03262699, ...,  0.24498899,
        -0.36573601,  0.328704  ]], dtype=float32)

In [17]:
# Save embeddings
np.save('final_embeddings_52', final_embeddings)

# Supervised Training

In [5]:
# Load embeddings from previous work
final_embeddings = np.load('./final_embeddings_52.npy')
word_dict = {}
with open('./data/embed_tweets_en_590M_52D_data/vocabulary_dict_52.pickle', 'rb') as myfile:
    word_dict = pickle.load(myfile)
# with open('./data/embed_tweets_en_200M_200D/vocabulary.pickle', 'rb') as myfile:
#     word_dict = pickle.load(myfile)

In [6]:
#Load label data
x_train_sentence, y_train, x_test_sentence, y_test = load_data_and_labels('./data/supervised_data/en_full.tsv.txt', './data/supervised_data/en_test.tsv')
print(len(x_test_sentence))
x_train, x_test = process_tweet(x_train_sentence, x_test_sentence, final_embeddings.shape[0])

20632
53
["there's", 'a', 'lot', 'of', 'stupid', '$', 'h', '!', 't', 'out', 'there', ',', 'but', 'polling', 'trump', 'v', 'kanye', 'west', 'may', 'take', 'the', 'cake', '.', 'all', 'i', 'can', 'think', 'to', 'say', 'is', ':', '#', '$', '%', '#', '$', '%', '$', '#', '%', '#', '$', '%', '#', '$', '%', '#', '$', '#', '$', '#', '$', '%']


In [7]:
# put Word2Vec on 590 million English Tweets using 52 dimensions.
vocabulary_size = final_embeddings.shape[0]
embedding_size = final_embeddings.shape[1]  # Dimension of the embedding vector.
graph = tf.Graph()

sequence_length=x_train.shape[1]
num_classes=3

# filter_sizes: The number of words we want our convolutional filters to cover. 
# We will have num_filters for each size specified here. 
# For example, [3, 4, 5] means that we will have filters that slide over 3, 4 and 5 words respectively, for a total of 3 * num_filters filters.
first_filter_sizes = [4]
first_pool_window_sizes = [4]
first_pool_strides = [2]


second_filter_window_sizes = [3]
num_filters = 200

# No L2 norm
l2_reg_lambda=0.0

with graph.as_default():
    input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
    input_y = tf.placeholder(tf.int64, [None], name="input_y")
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(tf.constant(0.0, shape=[vocabulary_size, embedding_size]),
                        trainable=False, name="embedding")

        embedding_placeholder = tf.placeholder(tf.float32, [vocabulary_size, embedding_size], name='word_embedding_placeholder')
        embedding_init = embeddings.assign(embedding_placeholder)  # assign exist word embeddings

        embedded_chars = tf.nn.embedding_lookup(embeddings, input_x)
    embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)
    print(input_x.shape)
    print(embedded_chars_expanded.shape)
    # Keeping track of l2 regularization loss (optional)
    l2_loss = tf.constant(0.0)
    
     # Add variable initializer.
    init = tf.global_variables_initializer()
    
    
    # Create first cnn : a convolution + maxpool layer for each filter size    
    # 1st Convolution Layer
    for i, first_filter_size in enumerate(first_filter_sizes):
        with tf.name_scope("conv-maxpool-1"):
            # Convolution Layer
            filter_shape = [first_filter_size, embedding_size, 1, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            print("CNN filter", W.shape)
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(
                embedded_chars_expanded,
                W,
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="conv")
            # Apply nonlinearity
            h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            # Maxpooling over the outputs and transform for next layer, so the "channel" of convolution 
            # will become "input_width" for next layer
#             pooled = tf.transpose(tf.nn.max_pool(
#                 h,
#                 ksize=[1, first_pool_window_sizes[i], 1, 1],
#                 strides=[1, first_pool_strides[i], 1, 1],
#                 padding='VALID',
#                 name="pool"), perm=[0, 1, 3, 2])
            pooled = tf.nn.max_pool(
                h,
                ksize=[1, first_pool_window_sizes[i], 1, 1],
                strides=[1, first_pool_strides[i], 1, 1],
                padding='VALID',
                name="pool")
#     print("conv1", conv.shape)
#     print("h1", h.shape)
#     print("pooled1", pooled_1.shape)
    
    # 2nd Convolutional Layer
#     for i, second_filter_size in enumerate(second_filter_window_sizes):
#         with tf.name_scope("conv-maxpool-2"):
#             # Convolution Layer
#             filter_shape = [second_filter_size, num_filters, 1, num_filters]
#             W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
#             print("CNN filter", W.shape)
#             b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
#             conv = tf.nn.conv2d(
#                 pooled,
#                 W,
#                 strides=[1, 1, 1, 1],
#                 padding="VALID",
#                 name="conv")
#             # Apply nonlinearity
#             h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
#             # Maxpooling over the outputs and transform for next layer, so the "channel" of convolution 
#             # will become "input_width" for next layer
#             pooled = tf.nn.max_pool(
#                 h,
#                 ksize=[1, h.shape[1], 1, 1],
#                 strides=[1, 1, 1, 1],
#                 padding='VALID',
#                 name="pool")
    for i, second_filter_size in enumerate(second_filter_window_sizes):
        with tf.name_scope("conv-maxpool-2"):
            # Convolution Layer
            filter_shape = [second_filter_size, 1, num_filters, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            print("CNN filter", W.shape)
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(
                pooled,
                W,
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="conv")
            # Apply nonlinearity
            h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            print(h.shape)
            # Maxpooling over the outputs and transform for next layer, so the "channel" of convolution 
            # will become "input_width" for next layer
            pooled = tf.nn.max_pool(
                h,
                ksize=[1, h.shape[1], 1, 1],
                strides=[1, 1, 1, 1],
                padding='VALID',
                name="pool")
 

    h_pool_flat = tf.reshape(pooled, [-1, num_filters])  # flatten pooling layers
    print("h_pool_flat", h_pool_flat.shape)
    
    # Add dropout
#     with tf.name_scope("dropout"):
#         self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

    # Final (unnormalized) scores and predictions
    
    # Fully connected hidden layer
    with tf.name_scope("hidden"):
        with tf.variable_scope("hidden"):
            W = tf.get_variable(
                "W",
                shape=[num_filters, num_filters],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            out = tf.nn.relu(tf.nn.xw_plus_b(h_pool_flat, W, b))
        
    
    with tf.name_scope("output"):
        with tf.variable_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            scores = tf.nn.xw_plus_b(out, W, b, name="scores")
            print("scores", scores.shape)
            predictions = tf.argmax(scores, 1, name="predictions")
            print("predictions", predictions.shape)


    # Calculate mean cross-entropy loss
    with tf.name_scope("loss"):
        losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=scores, labels=input_y)
        print("losses", losses.shape)
        loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

    # Accuracy
    with tf.name_scope("accuracy"):
        correct_predictions = tf.equal(predictions, input_y)
        accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
    

(?, 53)
(?, 53, 52, 1)
CNN filter (4, 52, 1, 200)
CNN filter (3, 1, 200, 200)
(?, 22, 1, 200)
h_pool_flat (?, 200)
scores (?, 3)
predictions (?,)
losses (?,)


In [8]:
show_graph(graph.as_graph_def())

In [ ]:
batch_size = 1024
num_epochs = 200

num_checkpoints = 5
print_train_every = 5
evaluate_every = 50
checkpoint_every = 10000000
allow_soft_placement=True
log_device_placement=False

with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=allow_soft_placement,
      log_device_placement=log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdadeltaOptimizer(1.0)
        grads_and_vars = optimizer.compute_gradients(loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", loss)
        acc_summary = tf.summary.scalar("accuracy", accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

#         # Write vocabulary
#         vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())


        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              input_x: x_batch,
              input_y: y_batch,
            }
            _, step, summaries, cur_loss, cur_accuracy, y_pred = sess.run(
                [train_op, global_step, train_summary_op, loss, accuracy, predictions],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
#             print(y_pred)
#             print(y_batch)
            if step % print_train_every == 0:
                f1 = f1_score(y_batch, y_pred, average = 'weighted')
                print("{}: step {}, loss {:g}, acc {:g}, f1 {:g}".format(time_str, step, cur_loss, cur_accuracy,
                                                                     f1))
            train_summary_writer.add_summary(summaries, step)



        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              input_x: x_batch,
              input_y: y_batch,
            }
            step, summaries, cur_loss, cur_accuracy, y_pred = sess.run(
                [global_step, dev_summary_op, loss, accuracy, predictions],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            f1 = f1_score(y_batch, y_pred, average = 'weighted')
            print("{}: step {}, loss {:g}, acc {:g}, f1 {:g}".format(time_str, step, cur_loss, cur_accuracy,
                                                                    f1))
            if writer:
                writer.add_summary(summaries, step)
                
        def dev_step_batch(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              input_x: x_batch,
              input_y: y_batch,
            }
            step, summaries, cur_loss, cur_accuracy, y_pred = sess.run(
                [global_step, dev_summary_op, loss, accuracy, predictions],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            f1 = f1_score(y_batch, y_pred, average = 'weighted')
#             print("{}: step {}, loss {:g}, acc {:g}, f1 {:g}".format(time_str, step, cur_loss, cur_accuracy,
#                                                                     f1))
            if writer:
                writer.add_summary(summaries, step)
            return cur_loss, cur_accuracy, f1

        
        sess.run(embedding_init, feed_dict={embedding_placeholder: final_embeddings})
        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), batch_size, num_epochs)
        
        batches_test = list(batch_iter(
            list(zip(x_test, y_test)), batch_size, 1))
        
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            
            current_step = tf.train.global_step(sess, global_step)
            if current_step % evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_test, y_test, writer=dev_summary_writer)
                print("")
            if current_step % checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

INFO:tensorflow:Summary name conv-maxpool-1/W:0/grad/hist is illegal; using conv-maxpool-1/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-1/W:0/grad/sparsity is illegal; using conv-maxpool-1/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-1/b:0/grad/hist is illegal; using conv-maxpool-1/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-1/b:0/grad/sparsity is illegal; using conv-maxpool-1/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-2/W:0/grad/hist is illegal; using conv-maxpool-2/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-2/W:0/grad/sparsity is illegal; using conv-maxpool-2/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-2/b:0/grad/hist is illegal; using conv-maxpool-2/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-2/b:0/grad/sparsity is illegal; using conv-maxpool-2/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name hidden/W:0/grad/hist is ill

/home/phejimlin/anaconda3/envs/tensorflow_3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


2017-11-19T01:34:08.325129: step 5, loss 1.0872, acc 0.416992, f1 0.275571
2017-11-19T01:34:08.620653: step 10, loss 1.01904, acc 0.446289, f1 0.406248
2017-11-19T01:34:08.921894: step 15, loss 1.04824, acc 0.410156, f1 0.283891
Current epoch:  1
2017-11-19T01:34:09.414834: step 20, loss 1.05712, acc 0.432617, f1 0.264631
2017-11-19T01:34:09.716680: step 25, loss 1.04282, acc 0.443359, f1 0.401082
2017-11-19T01:34:10.021661: step 30, loss 0.999571, acc 0.466797, f1 0.412625
2017-11-19T01:34:10.322073: step 35, loss 0.974774, acc 0.491211, f1 0.44286
Current epoch:  2
2017-11-19T01:34:10.599683: step 40, loss 1.08746, acc 0.415039, f1 0.25506
2017-11-19T01:34:10.906662: step 45, loss 1.0668, acc 0.420898, f1 0.253899
2017-11-19T01:34:11.237394: step 50, loss 0.972183, acc 0.478516, f1 0.412457

Evaluation:
2017-11-19T01:34:11.918859: step 50, loss 1.01784, acc 0.427443, f1 0.358912

Current epoch:  3
2017-11-19T01:34:12.258805: step 55, loss 0.995432, acc 0.445312, f1 0.340421
2017-11-1

2017-11-19T01:34:38.231304: step 450, loss 0.935649, acc 0.532861, f1 0.53737

Current epoch:  25
2017-11-19T01:34:38.536918: step 455, loss 0.842692, acc 0.628906, f1 0.627046
2017-11-19T01:34:38.829818: step 460, loss 0.849764, acc 0.601562, f1 0.576258
2017-11-19T01:34:39.131532: step 465, loss 0.925045, acc 0.532227, f1 0.505834
Current epoch:  26
2017-11-19T01:34:39.410210: step 470, loss 0.818555, acc 0.62207, f1 0.605461
2017-11-19T01:34:39.709668: step 475, loss 0.909168, acc 0.567383, f1 0.519754
2017-11-19T01:34:40.027448: step 480, loss 0.84074, acc 0.56543, f1 0.538754
2017-11-19T01:34:40.332590: step 485, loss 0.831996, acc 0.601562, f1 0.589298
Current epoch:  27
2017-11-19T01:34:40.640314: step 490, loss 0.93132, acc 0.530273, f1 0.456798
2017-11-19T01:34:40.999655: step 495, loss 0.820706, acc 0.604492, f1 0.590872
2017-11-19T01:34:41.320017: step 500, loss 0.858681, acc 0.587891, f1 0.578859

Evaluation:
2017-11-19T01:34:41.575994: step 500, loss 0.892872, acc 0.579633

2017-11-19T01:35:06.804668: step 895, loss 0.809379, acc 0.584961, f1 0.546412
2017-11-19T01:35:07.083574: step 900, loss 0.757914, acc 0.649371, f1 0.620473

Evaluation:
2017-11-19T01:35:07.319983: step 900, loss 0.977037, acc 0.502423, f1 0.483578

Current epoch:  50
2017-11-19T01:35:07.657310: step 905, loss 0.739031, acc 0.677734, f1 0.668053
2017-11-19T01:35:07.925771: step 910, loss 0.87332, acc 0.544922, f1 0.50479
2017-11-19T01:35:08.225261: step 915, loss 0.823661, acc 0.636719, f1 0.591359
Current epoch:  51
2017-11-19T01:35:08.522632: step 920, loss 0.747442, acc 0.665039, f1 0.649768
2017-11-19T01:35:08.844260: step 925, loss 0.721876, acc 0.683594, f1 0.680462
2017-11-19T01:35:09.143141: step 930, loss 0.78185, acc 0.605469, f1 0.571444
2017-11-19T01:35:09.460586: step 935, loss 0.781376, acc 0.658203, f1 0.61915
Current epoch:  52
2017-11-19T01:35:09.732593: step 940, loss 0.734579, acc 0.695312, f1 0.697701
2017-11-19T01:35:10.027345: step 945, loss 0.719136, acc 0.66699

...

2017-11-15T23:56:43.690551: step 34235, loss 0.00275748, acc 0.99707, f1 0.99707
Current epoch:  1902
2017-11-15T23:56:44.183898: step 34240, loss 0.00204252, acc 0.999023, f1 0.999023
2017-11-15T23:56:44.708401: step 34245, loss 0.00382085, acc 0.99707, f1 0.997074
2017-11-15T23:56:45.232923: step 34250, loss 0.00297496, acc 0.99707, f1 0.997069

Evaluation:
2017-11-15T23:56:45.912799: step 34250, loss 2.3715, acc 0.59209, f1 0.587197

Current epoch:  1903
2017-11-15T23:56:46.417224: step 34255, loss 0.000832774, acc 0.999023, f1 0.999022
2017-11-15T23:56:46.941658: step 34260, loss 0.00109041, acc 0.999023, f1 0.999023
2017-11-15T23:56:47.476139: step 34265, loss 0.00201358, acc 0.998047, f1 0.998046
2017-11-15T23:56:48.007591: step 34270, loss 0.00113494, acc 0.999023, f1 0.999025
Current epoch:  1904
2017-11-15T23:56:48.510465: step 34275, loss 4.54996e-05, acc 1, f1 1
2017-11-15T23:56:49.045011: step 34280, loss 0.000774745, acc 0.999023, f1 0.999023
2017-11-15T23:56:49.574508: st

2017-11-15T23:57:34.175152: step 34660, loss 0.00188707, acc 0.998047, f1 0.998046
2017-11-15T23:57:34.702596: step 34665, loss 0.00175806, acc 0.998047, f1 0.998047
Current epoch:  1926
2017-11-15T23:57:35.195943: step 34670, loss 0.00167867, acc 1, f1 1
2017-11-15T23:57:35.740932: step 34675, loss 4.38102e-05, acc 1, f1 1
2017-11-15T23:57:36.268894: step 34680, loss 0.00164087, acc 0.998047, f1 0.998048
2017-11-15T23:57:36.800436: step 34685, loss 0.000974823, acc 0.999023, f1 0.999022
Current epoch:  1927
2017-11-15T23:57:37.300805: step 34690, loss 0.00263804, acc 0.999023, f1 0.999024
2017-11-15T23:57:37.824735: step 34695, loss 0.00122397, acc 1, f1 1
2017-11-15T23:57:38.346161: step 34700, loss 0.00256147, acc 0.99707, f1 0.997071

Evaluation:
2017-11-15T23:57:39.016511: step 34700, loss 2.32407, acc 0.584044, f1 0.584504

Current epoch:  1928
2017-11-15T23:57:39.517441: step 34705, loss 0.00268897, acc 0.999023, f1 0.999023
2017-11-15T23:57:40.049395: step 34710, loss 4.29851e-

2017-11-15T23:58:25.107635: step 35095, loss 0.000995744, acc 0.999023, f1 0.999024
2017-11-15T23:58:25.610054: step 35100, loss 0.00443643, acc 0.995283, f1 0.995283

Evaluation:
2017-11-15T23:58:26.287908: step 35100, loss 2.3926, acc 0.574641, f1 0.574447

Current epoch:  1950
2017-11-15T23:58:26.812356: step 35105, loss 0.00232507, acc 0.999023, f1 0.999023
2017-11-15T23:58:27.339797: step 35110, loss 0.00133155, acc 1, f1 1
2017-11-15T23:58:27.863229: step 35115, loss 0.00213038, acc 0.998047, f1 0.998047
Current epoch:  1951
2017-11-15T23:58:28.357078: step 35120, loss 0.0039972, acc 0.999023, f1 0.999023
2017-11-15T23:58:28.888531: step 35125, loss 6.42001e-05, acc 1, f1 1
2017-11-15T23:58:29.406948: step 35130, loss 4.24269e-05, acc 1, f1 1
2017-11-15T23:58:29.942470: step 35135, loss 0.00151535, acc 0.999023, f1 0.999023
Current epoch:  1952
2017-11-15T23:58:30.451362: step 35140, loss 0.00317315, acc 0.99707, f1 0.997069
2017-11-15T23:58:30.981309: step 35145, loss 0.00167158

2017-11-15T23:59:16.047441: step 35525, loss 0.003397, acc 0.996094, f1 0.996097
2017-11-15T23:59:16.573172: step 35530, loss 0.00261285, acc 0.99707, f1 0.99707
Current epoch:  1974
2017-11-15T23:59:17.070532: step 35535, loss 0.00128627, acc 1, f1 1
2017-11-15T23:59:17.598976: step 35540, loss 0.00305306, acc 0.99707, f1 0.99707
2017-11-15T23:59:18.122408: step 35545, loss 0.00219351, acc 0.998047, f1 0.998051
2017-11-15T23:59:18.618262: step 35550, loss 4.03077e-05, acc 1, f1 1

Evaluation:
2017-11-15T23:59:19.275559: step 35550, loss 2.32959, acc 0.583268, f1 0.58375

Current epoch:  1975
2017-11-15T23:59:19.792471: step 35555, loss 0.000779976, acc 0.999023, f1 0.999023
2017-11-15T23:59:20.317908: step 35560, loss 0.00468215, acc 0.995117, f1 0.995117
2017-11-15T23:59:20.836826: step 35565, loss 0.00181298, acc 0.998047, f1 0.998048
Current epoch:  1976
2017-11-15T23:59:21.328169: step 35570, loss 0.000673392, acc 1, f1 1
2017-11-15T23:59:21.843578: step 35575, loss 0.000552767, a

2017-11-16T00:00:06.181691: step 35955, loss 0.00277737, acc 0.99707, f1 0.99707
2017-11-16T00:00:06.701612: step 35960, loss 4.48798e-05, acc 1, f1 1
Current epoch:  1998
2017-11-16T00:00:07.179440: step 35965, loss 0.000729787, acc 0.999023, f1 0.999023
2017-11-16T00:00:07.696354: step 35970, loss 0.00219639, acc 0.998047, f1 0.998046
2017-11-16T00:00:08.209256: step 35975, loss 0.00266371, acc 0.99707, f1 0.997069
2017-11-16T00:00:08.727171: step 35980, loss 4.40683e-05, acc 1, f1 1
Current epoch:  1999
2017-11-16T00:00:09.208988: step 35985, loss 0.00210667, acc 0.998047, f1 0.998047
2017-11-16T00:00:09.730914: step 35990, loss 0.00159984, acc 0.998047, f1 0.998048
2017-11-16T00:00:10.246324: step 35995, loss 0.00105196, acc 0.999023, f1 0.999023
2017-11-16T00:00:10.738167: step 36000, loss 0.00331172, acc 0.996855, f1 0.996856

Evaluation:
2017-11-16T00:00:11.392957: step 36000, loss 2.36229, acc 0.580215, f1 0.580538

